## Runnable 的调用机制

### Runnable 基类实现了 __call__ 方法

几乎所有 illufly 的类将 Runnable 作为基类。

Runnable 自己实现了 __call__ 方法，并在这个方法中调用 call 方法。<br>
因为 call 是一个抽象方法，因此要求子类必须实现自己它。

有了 __call__ 方法，你就可以将类的实例当作方法一样使用。

**这样做的好处：**
这很方便，也足够简单，因为你只需要记住 illufly 中的智能体对象只有一个方法，并且你不需要记住名字。

In [1]:
from illufly.types import Runnable

class MyRun(Runnable):
    def call(*args, **kwargs):
        return "hi"

r = MyRun()
r()

hi


这样，Runnable 就可以通过 __call__ 方法调用自己的 call 方法。

### 使用 EventBlock 交换事件信息流

In [2]:
from illufly.types import Runnable, EventBlock

class MyRun(Runnable):
    def call(*args, **kwargs):
        yield EventBlock("chunk", "hi")

r = MyRun()
r()

hi

## 使用 Runnable 的流式输出

### 默认的 log 处理

默认情况下，Runnable 的 handlers 列表中已经有 log 函数。
因此你可以隐藏调用 log 来打印流式内容。

In [3]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM

llm = FakeLLM()
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

In [4]:
# 这与下面的代码等价
from illufly.io import log

llm = FakeLLM(handlers=[log])
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

In [5]:
FakeLLM.allowed_params()

{'response': '响应内容',
 'sleep': '睡眠时间',
 'end_chk': '是否在最后输出一个 EndBlock',
 'start_marker': '开始标记，默认为 ```',
 'end_marker': '结束标记，默认为 ```',
 'final_answer_prompt': '最终答案提示词，可通过修改环境变量 ILLUFLY_FINAL_ANSWER_PROMPT 修改默认值',
 'func': '用于自定义工具的同步执行函数',
 'async_func': '用于自定义工具的异步执行函数',
 'name': '工具名称',
 'handlers': 'EventBlock 迭代器处理函数列表，默认为 [log]，当调用 call 方法时，会使用该列表中的函数逐个处理 EventBlock',
 'threads_group': '如果由 illufly 管理线程池实现并发或异步，则可以指定线程组名称，默认为 DEFAULT',
 'providers': '实例的 consumer_dict 属性由 providers 列表中每个 Runnable 的 provider_dict 属性提供',
 'consumers': '实例的 provider_dict 属性将被 consumers 列表中每个 Runnable 引用',
 'dynamic_providers': '如果实例在不同周期中重复使用，可能会希望先在绑定前先清除旧的绑定，此时就应该使用动态绑定，即执行 bind_provider 时提供 dynamic=True 参数',
 'lazy_binding_map': '有时你无法确定被哪个对象绑定，但能确定绑定映射，此时就可以使用 lazy_binding_map 参数，在绑定时由对方根据该参数进行绑定',
 'description': '工具描述',
 'tool_params': '工具参数',
 'knowledge': '待检索的资料或向量数据库',
 'team': '所属团队',
 'tools': '工具列表',
 'tools_handlers': '工具处理器列表',
 'tools_behavior': '工具处理行为, 包括 parse-execute, parse-exe

### 异步处理的 handler

只要 handler 中有一个是异步处理器，你就必须使用异步处理。<br>
alog 是 log 的异步版本。

In [6]:
from illufly.chat import FakeLLM
from illufly.io import alog

llm = FakeLLM(handlers=[alog])
await llm("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[CALLING] 873ac938-832f-4abc-b4db-63198a86fe7e
[USER] 你能帮我写一首关于兔子做梦的四句儿歌?
[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

### `illufly.io` 中的 log 和 usage

usage 处理函数用于捕捉生成器结果中的 

In [1]:
from illufly.chat import ChatOpenAI
from illufly.io import log, usage

openai = ChatOpenAI(handlers=[log, usage])
openai("你能帮我写一首关于兔子做梦的四句儿歌?", verbose=True)

[CALLING] b7c7f5c0-14e4-4ea9-9539-397bd505acdc
[USER] 你能帮我写一首关于兔子做梦的四句儿歌?
[INFO] 记住 10 轮对话
兔子梦里蹦跳快，  
甜甜满地都是胡萝卜。  
白日梦中游仙境，  
轻轻一笑唤醒梦。
[USAGE] {"prompt_tokens": 35, "completion_tokens": 62, "total_tokens": 97}
{"block_type": "usage", "content": "{\"prompt_tokens\": 35, \"completion_tokens\": 62, \"total_tokens\": 97}", "created_at": "2024-11-05T16:40:09.650547", "calling_info": {"request_id": null, "input": {"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "你能帮我写一首关于兔子做梦的四句儿歌?"}], "tools": null, "stream": true, "stream_options": {"include_usage": true}}, "output": [{"chunk": "兔"}, {"chunk": "子"}, {"chunk": "梦"}, {"chunk": "里"}, {"chunk": "蹦"}, {"chunk": "跳"}, {"chunk": "快"}, {"chunk": "，"}, {"chunk": "  \n"}, {"chunk": "甜"}, {"chunk": "甜"}, {"chunk": "满"}, {"chunk": "地"}, {"chunk": "都"}, {"chunk": "是"}, {"chunk": "胡"}, {"chunk": "萝"}, {"chunk": "卜"}, {"chunk": "。"}, {"chunk": "  \n"}, {"chunk": "白"}, {"chunk": "日"}, {"chunk": "梦"}, {"chunk": "中"}, {"chunk": "游"}, {"ch

'兔子梦里蹦跳快，  \n甜甜满地都是胡萝卜。  \n白日梦中游仙境，  \n轻轻一笑唤醒梦。'

## 使用绑定机制传播 handlers

In [1]:
from illufly.types import Runnable
from illufly.chat import FakeLLM
from illufly.io import log

### 定义一个新的 handler

定义 `handler` 必须使用形如 `block,, verbose, **kwargs` 这样的入参结构：

In [5]:
def myhandler(block, verbose=False, **kwargs):
    if block.block_type == 'info':
        print("I got a [info] event, but I wont tell you...")

In [6]:
llm = FakeLLM()
llm("hi")

Reply >> hi


'Reply >> hi'

### 将 provider 中的 handler 传递给 consumer

In [4]:
class myrun(Runnable):
    def call(self, prompt, **kwargs):
        llm = FakeLLM()
        self.bind_consumer(llm)
        resp = llm(prompt)
r = myrun(handlers = [log, myhandler])
r("hi")

I got a [info] event, but I wont tell you...
这是一个模拟调用!

In [5]:
r.consumer_tree

{'provider': <Runnable myrun.4767904816>,
 'consumer_tree': [(<Runnable FakeLLM.4767904144>,
   {},
   {'provider': <Runnable FakeLLM.4767904144>, 'consumer_tree': []})]}

In [8]:
llm.provider_tree

{'consumer': <Runnable FakeLLM.4767898768>, 'provider_tree': []}

## EventsTree

Runnable 自带了一个事件树，汇聚所有收集到的 EventBlock 数据。

In [1]:
# 使用 handler 函数的另一种方式：
from illufly.chat import FakeLLM

llm = FakeLLM()
llm("你能帮我写一首关于兔子做梦的四句儿歌?")

Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?


'Reply >> 你能帮我写一首关于兔子做梦的四句儿歌?'

In [2]:
llm.events_tree

{'name': 'FakeLLM.4380061712',
 'type': 'FakeLLM',
 'events': [EventBlock(block_type=<calling>, content=<10aefa7c-6314-46e6-bc7c-264dc753d850>),
  EventBlock(block_type=<user>, content=<你能帮我写一首关于兔子做梦的四句儿歌?>),
  EventBlock(block_type=<info>, content=<记住 10 轮对话>),
  EventBlock(block_type=<info>, content=<I am FakeLLM>),
  EventBlock(block_type=<chunk>, content=<R>),
  EventBlock(block_type=<chunk>, content=<e>),
  EventBlock(block_type=<chunk>, content=<p>),
  EventBlock(block_type=<chunk>, content=<l>),
  EventBlock(block_type=<chunk>, content=<y>),
  EventBlock(block_type=<chunk>, content=< >),
  EventBlock(block_type=<chunk>, content=<>>),
  EventBlock(block_type=<chunk>, content=<>>),
  EventBlock(block_type=<chunk>, content=< >),
  EventBlock(block_type=<chunk>, content=<你>),
  EventBlock(block_type=<chunk>, content=<能>),
  EventBlock(block_type=<chunk>, content=<帮>),
  EventBlock(block_type=<chunk>, content=<我>),
  EventBlock(block_type=<chunk>, content=<写>),
  EventBlock(block_typ